Data Date Range:
    <br />Start time: 2016-05-05 09:40:26
    <br />End time: 2018-01-17 01:59:42

This notebook gets all actions on the trailer modal before 2018-01-17 01:59:42. Researchers are excluded.  
    
#### This notebook generates the following table:
- log_trailer_actions_sessionIds 

Users in each login can open the trailer modal many times. Each time they open the modal, we call it a new session and assign a new session ID. 

Session ID is helpful to match the recommendations a user received with information about the recommendations, especially their age (which is the time difference between film release date and date when they were recommended) 

### Import ###

In [1]:
import numpy as np
import pandas as pd

### Load the dataset with specific actions in each login session in the experiment

- The variable 'action_data' stores the table with logins, trailers, user actions, and is used in many other notebooks, so we create this dataset in this notebook.

- We've excluded movielens users from this table


In [2]:
fields = ['userId', 'loginId','movieId','positionInList','action','logJson']

### Connect to database

In [4]:
# 2.23.2018 Update: removed grouplens researchers

# Connect to the database
import pymysql.cursors
import json
import re

# We will use these lists to generate a table later. 

sessionIds = []
loginIds = []

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_mirror")

try:
    with connection.cursor() as cursor:
        # This query select rows from log_trailer_actions where the action is either TrailerModalLaunched or Recommendations Refreshed
        # These two actions indicate that a recommendation list is generated. 
        
        sql = """
        SELECT log_trailer_actions.userId, log_trailer_actions.loginId, log_trailer_actions.movieId, log_trailer_actions.action, log_trailer_actions.tstamp, log_trailer_actions.positionInList        
        FROM user_data
        RIGHT JOIN log_trailer_actions
        ON log_trailer_actions.userId = user_data.userId
        WHERE log_trailer_actions.tstamp < '2018-01-17 01:59:42' AND user_data.excludeFromExperiments='N'
        ORDER BY log_trailer_actions.loginId, log_trailer_actions.tstamp
        """
        cursor.execute(sql)
        loginIds.append(0)
        sessionId = 1
            
        # Iterate each row in the query result
        for row in cursor:
            loginIds.append(row[1])
            if (row[3] == 'TrailerModalLaunched'):
                # If a new login, set sessionId = 1
                if (loginIds[len(loginIds)-2] != row[1]):
                    sessionId = 1
#                     print(row)
                # else, everytime a user launches the modal, increment sessionId by 1
                else:
                    sessionId = sessionId+1
                    
            sessionIds.append(sessionId)
            
        cursor.execute(sql)
        result = cursor.fetchall()
            
finally: 
    connection.close()


In [5]:
loginIds[0:10]

[0,
 '008C57f',
 '008C57f',
 '008C57f',
 '008C57f',
 '008C57f',
 '008C57f',
 '008C57f',
 '009NNgE',
 '009NNgE']

In [6]:
print(len(sessionIds))
print(len(loginIds))

1237423
1237424


In [7]:
# Removie dummy loginId
loginIds = loginIds[1:]

In [8]:
action_data = pd.DataFrame(list(result), columns=['userId','loginId','movieId','action','tstamp','position'])

In [9]:
action_data['sessionIds'] = sessionIds

In [10]:
action_data.head()

,userId,loginId,movieId,action,tstamp,position,sessionIds
0,276159,008C57f,118985,PlayedFromLaunch,2016-09-12 13:53:04,0,1
1,276159,008C57f,118985,TrailerModalLaunched,2016-09-12 13:53:04,0,2
2,276159,008C57f,118985,TrailerModalShutdown,2016-09-12 13:55:31,0,2
3,276159,008C57f,157140,TrailerModalLaunched,2016-09-12 13:55:52,0,3
4,276159,008C57f,157140,PlayedFromLaunch,2016-09-12 13:55:53,0,3


In [11]:
# action_data.to_csv('../Clean_Data/log_trailer_actions_nojson.csv')

# New dataframe with sessionIDs
action_data.to_csv('../Clean_Data/log_trailer_actions_sessionIds.csv')